In [1]:
import tensorflow as tf
tf.enable_eager_execution()
tfe = tf.contrib.eager
import numpy as np
import os

/Users/cengjianhuan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# create a log directory
logdir = os.mkdir("/Users/cengjianhuan/Downloads/logs")

In [2]:
mnist = tf.keras.datasets.mnist

# Dataset will be cached locally after you run this code
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# These types are required for the operation we use to compute
# loss. Omit, and you shall receive a cryptic error message.
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)



buffer_size = 5000
batch_size = 100

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size)
train_dataset = train_dataset.batch(batch_size)

In [3]:
def loss(logits, labels):
      # FIX ME
      # You will need to modify this function, of course.
      # Best bet, use tf.nn.sparse_softmax_cross_entropy_with_logits
      # though if you're interested, you can write your own.
      return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels ))

In [4]:
def compute_accuracy(logits, labels):
    # You shoud not need to modify this function
    predictions = tf.argmax(logits, axis=1)
    batch_size = int(logits.shape[0])
    return tf.reduce_sum(tf.cast(tf.equal(predictions, labels), dtype=tf.float32)) / batch_size

In [5]:
def train(model, images, labels):
    # You should not need to modify this function
    with tf.GradientTape() as tape:
        logits = model(images)
        loss_value = loss(logits, labels)  
    grads = tape.gradient(loss_value, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables))
    return loss_value

In [6]:
# each experiment
def iexperiment(model,logdir_i,epochs):    
    step_counter = 0
    writer = tf.contrib.summary.create_file_writer(logdir=logdir_i, flush_millis=1000)
    with writer.as_default():
        with tf.contrib.summary.always_record_summaries():
            for epoch_n in range(epochs):
                print('Epoch #%d' % (epoch_n))
                for (batch, (images, labels)) in enumerate(train_dataset):
                    loss_value = train(model, images, labels)
                    tf.contrib.summary.scalar("loss", loss_value, step=step_counter)
                    step_counter +=1
                    test_accuracy = compute_accuracy(model(x_test), y_test)
                print('Step #%d\tLoss: %.4f' % (step_counter, loss_value))
                print('Accuracy #%.2f\n' % (test_accuracy))    
    return loss_value, test_accuracy

In [7]:
# a small deep model for compared experiments
class b_Deep_Model(tf.keras.Model):
    def __init__(self, act, kinit):
        super(b_Deep_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(156, activation=act, kernel_initializer=kinit)
        self.dropout = tf.keras.layers.Dropout(0.25)
        self.dense2 = tf.keras.layers.Dense(10,activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)  
        x = self.dense2(x)
        return x

# Q3: The linear model

In [7]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        # self.conv = tf.keras.layers.Conv2D(filters=64,kernel_size=(5,5),padding='same',activation='elu')
        # self.maxpooling = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        # self.dropout = tf.keras.layers.Dropout(0.25)
        self.dense = tf.keras.layers.Dense(10,activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        # x = self.conv(x)
        # x = self.maxpooling(x)
        # x = self.dropout(x)  
        x = self.dense(x)
        return x

In [16]:
# The first time you run the below block it will crash
# with an error 'ValueError: No variables provided.''
# This is because the call method of your model
# is not using any trainable variables.
# (As written, the model just flattens the images.)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
model = Model()

epochs = 10
step_counter = 0

for epoch_n in range(epochs):
  print('Epoch #%d' % (epoch_n))
  for (batch, (images, labels)) in enumerate(train_dataset):
    loss_value = train(model, images, labels)
    step_counter +=1
  
    if step_counter % 100 == 0:
      print('Step #%d\tLoss: %.4f' % (step_counter, loss_value))
      #print(step_counter, loss_value)

  test_accuracy = compute_accuracy(model(x_test), y_test)
  print('Accuracy #%.2f\n' % (test_accuracy))

Epoch #0
Step #100	Loss: 2.0517
Step #200	Loss: 1.8722
Step #300	Loss: 1.8326
Step #400	Loss: 1.8214
Step #500	Loss: 1.7275
Step #600	Loss: 1.6722
Accuracy #0.81

Epoch #1
Step #700	Loss: 1.7354
Step #800	Loss: 1.7313
Step #900	Loss: 1.7221
Step #1000	Loss: 1.6316
Step #1100	Loss: 1.6479
Step #1200	Loss: 1.6504
Accuracy #0.87

Epoch #2
Step #1300	Loss: 1.6884
Step #1400	Loss: 1.6326
Step #1500	Loss: 1.6777
Step #1600	Loss: 1.6363
Step #1700	Loss: 1.6351
Step #1800	Loss: 1.6023
Accuracy #0.89

Epoch #3
Step #1900	Loss: 1.6091
Step #2000	Loss: 1.6076
Step #2100	Loss: 1.6211
Step #2200	Loss: 1.6088
Step #2300	Loss: 1.5884
Step #2400	Loss: 1.6025
Accuracy #0.90

Epoch #4
Step #2500	Loss: 1.6333
Step #2600	Loss: 1.6056
Step #2700	Loss: 1.6471
Step #2800	Loss: 1.6553
Step #2900	Loss: 1.6510
Step #3000	Loss: 1.6057
Accuracy #0.90

Epoch #5
Step #3100	Loss: 1.5890
Step #3200	Loss: 1.6010
Step #3300	Loss: 1.6476
Step #3400	Loss: 1.5581
Step #3500	Loss: 1.6137
Step #3600	Loss: 1.5492
Accuracy #0

# Q4: visualization of the linear model

In [8]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
model = Model()
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_linear"
loss, test_acc = iexperiment(model,logdir_i,10)

Epoch #0
Step #600	Loss: 1.7530
Accuracy #0.81

Epoch #1
Step #1200	Loss: 1.6861
Accuracy #0.83

Epoch #2
Step #1800	Loss: 1.6575
Accuracy #0.83

Epoch #3
Step #2400	Loss: 1.6025
Accuracy #0.83

Epoch #4
Step #3000	Loss: 1.6238
Accuracy #0.84

Epoch #5
Step #3600	Loss: 1.6998
Accuracy #0.84

Epoch #6
Step #4200	Loss: 1.6327
Accuracy #0.88

Epoch #7
Step #4800	Loss: 1.5644
Accuracy #0.90

Epoch #8
Step #5400	Loss: 1.5788
Accuracy #0.90

Epoch #9
Step #6000	Loss: 1.5766
Accuracy #0.91



![title](img/loss_linear.png)

# Q5: the deep model

In [7]:
class Deep_Model(tf.keras.Model):
    def __init__(self):
        super(Deep_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        # self.conv = tf.keras.layers.Conv2D(filters=64,kernel_size=(5,5),padding='same',activation='elu')
        # self.maxpooling = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        self.dense1 = tf.keras.layers.Dense(256, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.25)
        self.dense2 = tf.keras.layers.Dense(10,activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        # x = self.conv(x)
        # x = self.maxpooling(x)
        x = self.dense1(x)
        x = self.dropout(x)  
        x = self.dense2(x)
        return x

In [10]:
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [8]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
model = Deep_Model()
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_deep"
loss, test_acc = iexperiment(model,logdir_i,10)

Epoch #0
Step #600	Loss: 1.6615
Accuracy #0.81

Epoch #1
Step #1200	Loss: 1.6212
Accuracy #0.83

Epoch #2
Step #1800	Loss: 1.5747
Accuracy #0.84

Epoch #3
Step #2400	Loss: 1.5681
Accuracy #0.84

Epoch #4
Step #3000	Loss: 1.5622
Accuracy #0.84

Epoch #5
Step #3600	Loss: 1.6371
Accuracy #0.85

Epoch #6
Step #4200	Loss: 1.6243
Accuracy #0.85

Epoch #7
Step #4800	Loss: 1.5792
Accuracy #0.85

Epoch #8
Step #5400	Loss: 1.6100
Accuracy #0.85

Epoch #9
Step #6000	Loss: 1.5610
Accuracy #0.85



![title](img/loss_deep.png)

# Q6.a: The learning rates in the deep model

In [8]:
r = 0.5

optimizer = tf.train.GradientDescentOptimizer(learning_rate=r)
kinit = 'glorot_uniform'
act = 'relu'

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6a_"+str(r)
loss, test_acc = iexperiment(model, logdir_i, 5)

Epoch #0
Step #600	Loss: 1.5685
Accuracy #0.92

Epoch #1
Step #1200	Loss: 1.4929
Accuracy #0.94

Epoch #2
Step #1800	Loss: 1.5163
Accuracy #0.94

Epoch #3
Step #2400	Loss: 1.4845
Accuracy #0.95

Epoch #4
Step #3000	Loss: 1.4848
Accuracy #0.95



In [8]:
r = 0.1

optimizer = tf.train.GradientDescentOptimizer(learning_rate=r)
kinit = 'glorot_uniform'
act = 'relu'

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6a_"+str(r)
loss, test_acc = iexperiment(model, logdir_i, 5)

Epoch #0
Step #600	Loss: 1.5603
Accuracy #0.89

Epoch #1
Step #1200	Loss: 1.5585
Accuracy #0.91

Epoch #2
Step #1800	Loss: 1.5365
Accuracy #0.92

Epoch #3
Step #2400	Loss: 1.5391
Accuracy #0.92

Epoch #4
Step #3000	Loss: 1.5052
Accuracy #0.93



In [8]:
r = 0.001

optimizer = tf.train.GradientDescentOptimizer(learning_rate=r)
kinit = 'glorot_uniform'
act = 'relu'

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6a_"+str(r)
loss, test_acc = iexperiment(model, logdir_i, 5)

Epoch #0
Step #600	Loss: 2.2949
Accuracy #0.13

Epoch #1
Step #1200	Loss: 2.2914
Accuracy #0.18

Epoch #2
Step #1800	Loss: 2.2799
Accuracy #0.21

Epoch #3
Step #2400	Loss: 2.2704
Accuracy #0.23

Epoch #4
Step #3000	Loss: 2.2520
Accuracy #0.27



In [8]:
r = 0.00001

optimizer = tf.train.GradientDescentOptimizer(learning_rate=r)
kinit = 'glorot_uniform'
act = 'relu'

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6a_"+str(r)
loss, test_acc = iexperiment(model, logdir_i, 5)

Epoch #0
Step #600	Loss: 2.3107
Accuracy #0.14

Epoch #1
Step #1200	Loss: 2.3075
Accuracy #0.14

Epoch #2
Step #1800	Loss: 2.3024
Accuracy #0.14

Epoch #3
Step #2400	Loss: 2.3122
Accuracy #0.14

Epoch #4
Step #3000	Loss: 2.2958
Accuracy #0.14



![title](img/learning_rates.png)
+ The learning rates in the experiments seems the higher, the better, which is not theoretically true. 
+ Rate 0.5 and 0.1 perform well with steady loss 1.50 and 1.55, respectively. 
+ But 0.1 and 0.5 are theoretically high rate.
+ Both 0.001 and 0.00001 act like theoretically low rate. Rate 0.00001 is too low as the experiments, but 0.001 is usually the reasonable choice.
+ Both the loss of 0.1 and 0.5 is slowing decreasing, far away from convergency.


# Q6.b: Different activation functions in the deep model

In [8]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
kinit = 'glorot_uniform'
act='tanh'  

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6b_"+act
loss, test_acc = iexperiment(model,logdir_i,5)

Epoch #0
Step #600	Loss: 1.6536
Accuracy #0.75

Epoch #1
Step #1200	Loss: 1.5933
Accuracy #0.90

Epoch #2
Step #1800	Loss: 1.5783
Accuracy #0.91

Epoch #3
Step #2400	Loss: 1.5464
Accuracy #0.92

Epoch #4
Step #3000	Loss: 1.5485
Accuracy #0.92



In [8]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
kinit = 'glorot_uniform'
act='sigmoid'  

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6b_"+act
loss, test_acc = iexperiment(model,logdir_i,5)

Epoch #0
Step #600	Loss: 2.0143
Accuracy #0.56

Epoch #1
Step #1200	Loss: 1.9043
Accuracy #0.58

Epoch #2
Step #1800	Loss: 1.8400
Accuracy #0.64

Epoch #3
Step #2400	Loss: 1.8122
Accuracy #0.66

Epoch #4
Step #3000	Loss: 1.8382
Accuracy #0.67



![title](img/acti.png)
+ Relu performs better than sigmoid and tanh as expected. 
+ Actually, tanh also performs good, where the gradient vanishing phenomenon does not occur for this test.
+ All converge in similar step(2k) with loss 1.6 for both Relu and tanh, loss 1.8 for sigmoid
  

# Q6.c: Different gradient descent optimizers

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
kinit = 'glorot_uniform'
act='sigmoid'  

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6c_"+str(1)
loss, test_acc = iexperiment(model,logdir_i,5)

Epoch #0
Step #600	Loss: 1.8979
Accuracy #0.56

Epoch #1
Step #1200	Loss: 1.8844
Accuracy #0.55

Epoch #2
Step #1800	Loss: 1.7992
Accuracy #0.56

Epoch #3
Step #2400	Loss: 1.9307
Accuracy #0.57

Epoch #4
Step #3000	Loss: 1.8147
Accuracy #0.64



In [8]:
optimizer = tf.train.MomentumOptimizer(learning_rate=0.1, momentum=0.7)
kinit = 'glorot_uniform'
act='sigmoid'  

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6c_"+str(2)
loss, test_acc = iexperiment(model,logdir_i,5)

Epoch #0
Step #600	Loss: 1.8358
Accuracy #0.66

Epoch #1
Step #1200	Loss: 1.8484
Accuracy #0.68

Epoch #2
Step #1800	Loss: 1.5784
Accuracy #0.83

Epoch #3
Step #2400	Loss: 1.6518
Accuracy #0.84

Epoch #4
Step #3000	Loss: 1.6160
Accuracy #0.85



![title](img/optimizer.png)
+ GradientDescentOptimizer performs the best, followed MomentumOptimizer and AdamOptimizer.
+ GradientDescentOptimizer is relatively steady a bit faster in the step of 1.3k with the lowest loss 1.55 than that of MomentumOptimizer in the step of 2.0k with a lower loss 1.61. 
+ AdamOptimizer is slowly decreasing until the end(to the loss 1.85)


# Q6.d: Different weight initialization strategies

In [8]:
optimizer = tf.train.MomentumOptimizer(learning_rate=0.1, momentum=0.7)
kinit = 'Orthogonal'
act='sigmoid'  

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6c_"+kinit
loss, test_acc = iexperiment(model,logdir_i,5)

Epoch #0
Step #600	Loss: 1.7609
Accuracy #0.66

Epoch #1
Step #1200	Loss: 1.7580
Accuracy #0.73

Epoch #2
Step #1800	Loss: 1.6988
Accuracy #0.76

Epoch #3
Step #2400	Loss: 1.6368
Accuracy #0.76

Epoch #4
Step #3000	Loss: 1.6496
Accuracy #0.76



In [8]:
optimizer = tf.train.MomentumOptimizer(learning_rate=0.1, momentum=0.7)
kinit = 'lecun_uniform'
act='sigmoid'  

model = b_Deep_Model(act, kinit)
logdir_i = "/Users/cengjianhuan/Downloads/logs/exp_6c_"+kinit
loss, test_acc = iexperiment(model,logdir_i,5)

Epoch #0
Step #600	Loss: 1.8506
Accuracy #0.67

Epoch #1
Step #1200	Loss: 1.7984
Accuracy #0.73

Epoch #2
Step #1800	Loss: 1.6778
Accuracy #0.76

Epoch #3
Step #2400	Loss: 1.6503
Accuracy #0.76

Epoch #4
Step #3000	Loss: 1.6343
Accuracy #0.82



![title](img/kernel_initializer.png)
+	weight initialization strategy 'glorot_uniform'performs the best while 'Orthogonal' and 'lecun_uniform' are similarly good. 

+	they are relatively steady in the similar step(about 1.5k), but the loss of 'glorot_uniform' is about 1.55 less than those of 'Orthogonal' and 'lecun_uniform'(about 1.7)

